In [1]:
# pose_detection_and_correction.py
import cv2
import mediapipe as mp
import numpy as np

In [2]:
# Initialize MediaPipe Pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [4]:

# Function for providing feedback based on pose landmarks
def provide_feedback(landmarks):
    feedback = []
    
    # Example: Check arm alignment (can be extended)
    shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
    elbow = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y])
    wrist = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])

    # Calculate angle between shoulder, elbow, and wrist
    angle = np.degrees(np.arccos(np.dot(shoulder - elbow, wrist - elbow) /
                                 (np.linalg.norm(shoulder - elbow) * np.linalg.norm(wrist - elbow))))

    # Feedback based on arm angle
    if angle < 160:
        feedback.append("Your arm is not fully extended. Try to straighten your arm.")
    else:
        feedback.append("Alignment looks good. Keep practicing!")

    return feedback

def detect_pose_and_correct(frame):
    # Convert the frame to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        feedback = provide_feedback(landmarks)

        # Draw pose landmarks
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display feedback on the frame
        cv2.putText(frame, " ".join(feedback), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    return frame


In [5]:

def start_pose_detection():
    # Open video capture (webcam)
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform pose detection and correction
        frame = detect_pose_and_correct(frame)

        # Show the frame
        cv2.imshow("Pose Detection & Correction", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
